In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Hi everyone,

Here i have used Tensorflow keras CNN model in this kernal to predict 5 class label in dataset for eye disease classification.

First of all import all the necessary libraries 

In [2]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten
from tensorflow.keras.layers import Conv2D,MaxPool2D
from tensorflow.keras.layers import Input, Dense
from keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

from tensorflow.keras.applications.vgg16 import VGG16
import pandas as pd
import numpy as np
import os
from numpy import argmax


Using TensorFlow backend.


Now here read all the directories available in eye disease folder

In [3]:
PATH = "/kaggle/input/eye-disease-dataset/Eye_diseases"
data_dir_list = os.listdir(PATH)
data_dir_list

['Bulging_Eyes', 'Crossed_Eyes', 'Glaucoma', 'Cataracts', 'Uveitis']

Now here we'll prepare data to feed into the model. first read all the images from the dataset and resize it with specific size and prepare all the image data list for multiple folder with  target column.

In [4]:
img_rows=224
img_cols=224
num_channel=3

num_epoch = 50
batch_size = 32

img_data_list=[]
classes_names_list=[]
target_column=[]
for dataset in data_dir_list:
    classes_names_list.append(dataset)
    print("Getting image from {} folder".format(dataset))
    img_list= os.listdir(PATH +"/"+ dataset)
    for img in img_list:
        input_img = cv2.imread(PATH +"/"+dataset+"/"+img)
        input_img_resize=cv2.resize(input_img,(img_rows,img_cols))
        img_data_list.append(input_img_resize)
        target_column.append(dataset)

Getting image from Bulging_Eyes folder
Getting image from Crossed_Eyes folder
Getting image from Glaucoma folder
Getting image from Cataracts folder
Getting image from Uveitis folder


In [5]:
num_classes = len(classes_names_list)
print("num_classes",num_classes)
img_data = np.array(img_data_list) # convert images in numpy array 
img_data = img_data.astype('float32')
img_data /= 255
print("Shape of image data",img_data.shape)
num_of_samples = img_data.shape[0]
input_shape = img_data[0].shape 
print("number of samples",num_of_samples)
print("target column before encoding",target_column)

num_classes 5
Shape of image data (383, 224, 224, 3)
number of samples 383
target column before encoding ['Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Bulging_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes', 'Crossed_Eyes'

Here encode the target column becuase we can't use string in model. so here we are using label encoder to encode target column.

In [6]:
Labelencoder = LabelEncoder()
target_column = Labelencoder.fit_transform(target_column)
np.unique(target_column)

array([0, 1, 2, 3, 4])

Now check the target column value after the encoding

In [7]:
target_column

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

Now lets split data with validation split equal to 30% from training data. Validation split is used to determine that our model is not geting over-fitted.

In [8]:
target_column_hotcoded = to_categorical(target_column,num_classes)
X,Y = shuffle(img_data,target_column_hotcoded,random_state=2)
X_train,X_temp,y_train,y_temp = train_test_split(X,Y,test_size=0.3,random_state=2)
X_test,X_val,y_test,y_val = train_test_split(X_temp,y_temp,test_size=0.3,random_state=2)

Create CNN Model to predict class label for eye disease

In [9]:
first_Mod = Sequential()

first_Mod.add(Conv2D(64,(3,3),activation='relu',input_shape=input_shape))
first_Mod.add(Conv2D(64,(3,3),activation='relu'))
first_Mod.add(MaxPool2D(pool_size=(2,2)))
first_Mod.add(Dropout(0.2))

first_Mod.add(Conv2D(128,(3,3),activation='relu'))
first_Mod.add(Conv2D(128,(3,3),activation='relu'))
first_Mod.add(MaxPool2D(pool_size=(2,2)))
first_Mod.add(Dropout(0.2))

first_Mod.add(Flatten())
first_Mod.add(Dense(128,activation='relu'))
first_Mod.add(Dropout(0.2))
first_Mod.add(Dense(num_classes,activation='softmax'))
first_Mod.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

first_Mod.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 106, 106, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 53, 53, 128)       0

Now fit the data in the model and evaluate the model accuracy and Loss.

In [10]:
hist = first_Mod.fit(X_train,y_train,batch_size=batch_size,epochs=num_epoch,verbose=1,validation_data=(X_test,y_test))
score = first_Mod.evaluate(X_test,y_test,batch_size=batch_size)
print('Test Loss',score[0])
print("Test Accuracy",score[1])

Train on 268 samples, validate on 80 samples
Epoch 1/50
268/268 [==============================] - 6s 22ms/sample - loss: 6.1953 - accuracy: 0.2537 - val_loss: 1.5533 - val_accuracy: 0.1500
Epoch 2/50
268/268 [==============================] - 1s 4ms/sample - loss: 1.5300 - accuracy: 0.2201 - val_loss: 1.4949 - val_accuracy: 0.4375
Epoch 3/50
268/268 [==============================] - 1s 4ms/sample - loss: 1.4858 - accuracy: 0.3246 - val_loss: 1.5387 - val_accuracy: 0.4375
Epoch 4/50
268/268 [==============================] - 1s 4ms/sample - loss: 1.4932 - accuracy: 0.4254 - val_loss: 1.4241 - val_accuracy: 0.4375
Epoch 5/50
268/268 [==============================] - 1s 4ms/sample - loss: 1.4020 - accuracy: 0.4440 - val_loss: 1.4177 - val_accuracy: 0.4375
Epoch 6/50
268/268 [==============================] - 1s 4ms/sample - loss: 1.4286 - accuracy: 0.3993 - val_loss: 1.4174 - val_accuracy: 0.4375
Epoch 7/50
268/268 [==============================] - 1s 4ms/sample - loss: 1.4100 - accur